In [76]:
import torch

In [77]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [78]:
%%writefile data_setup.py

import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

NUM_WORKERS = os.cpu_count()

def create_dataloader(train_dir: str,
                      test_dir: str,
                      transforms: transforms.Compose,
                      batch_size: int = 32,
                      num_workers: int = NUM_WORKERS):
    train_data = datasets.ImageFolder(root=train_dir, transform=transforms)
    test_data = datasets.ImageFolder(root=test_dir, transform=transforms)

    class_names = train_data.classes

    train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=num_workers)
    test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=num_workers)

    return train_dataloader, test_dataloader, class_names

Overwriting data_setup.py


In [79]:
import os
from pathlib import Path

train_dir = Path("data/train")
test_dir = Path("data/val")
train_dir, test_dir

(WindowsPath('data/train'), WindowsPath('data/val'))

In [80]:
%%writefile engine.py

import torch
from tqdm.auto import tqdm
from typing import Dict, List, Tuple

def train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               device: torch.device):
    train_loss, train_acc = 0, 0

    model.train()

    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        y_pred = model(X)

        loss = loss_fn(y_pred, y)
        train_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
        train_acc += (y_pred_class == y).sum().item() / len(y_pred)

    train_loss /= len(dataloader)
    train_acc /= len(dataloader)

    return train_loss, train_acc

def test_step(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              device: torch.device):
    test_loss, test_acc = 0, 0

    model.eval()
    with torch.inference_mode():
        for batch, (X, y) in enumerate(dataloader):
            X, y = X.to(device), y.to(device)

            test_pred = model(X)

            loss = loss_fn(test_pred, y)
            test_loss += loss.item()

            test_pred_labels = test_pred.argmax(dim=1)
            test_acc += (test_pred_labels == y).sum().item() / len(test_pred_labels)

    test_loss /= len(dataloader)
    test_acc /= len(dataloader)

    return test_loss, test_acc

def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          loss_fn: torch.nn.Module,
          optimizer: torch.optim.Optimizer,
          epochs: int,
          device: torch.device):
    results = {
        "train_loss": [],
        "train_acc": [],
        "test_loss": [],
        "test_acc": []
    }

    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model,
                                           dataloader=train_dataloader,
                                           loss_fn=loss_fn,
                                           optimizer=optimizer,
                                           device=device)

        test_loss, test_acc = test_step(model=model,
                                        dataloader=test_dataloader,
                                        loss_fn=loss_fn,
                                        device=device)

        print(f"Epoch: {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}")

        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

    return results

Overwriting engine.py


In [81]:
import torchvision
from torchvision import transforms

weights = torchvision.models.ResNet50_Weights.DEFAULT
weights

ResNet50_Weights.IMAGENET1K_V2

In [82]:
auto_transforms = weights.transforms()
auto_transforms

ImageClassification(
    crop_size=[224]
    resize_size=[232]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BILINEAR
)

In [83]:
import data_setup

train_dataloader, test_dataloader, class_names = data_setup.create_dataloader(train_dir=train_dir,
                                                                              test_dir=test_dir,
                                                                              transforms=auto_transforms,
                                                                              batch_size=32)

train_dataloader, test_dataloader, class_names

(<torch.utils.data.dataloader.DataLoader at 0x1ccf7fa7290>,
 ['MildDemented', 'ModerateDemented', 'NonDemented', 'VeryMildDemented'])

In [93]:
weights = torchvision.models.ResNet50_Weights.DEFAULT
model = torchvision.models.resnet50(weights=weights).to(device)
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [85]:
len(class_names)

4

In [101]:
for param in model.parameters():
    param.requires_grad = False

In [102]:
from torch import nn

model.fc = torch.nn.Sequential(
    nn.Dropout(p=0.2),
    nn.Linear(in_features=2048, out_features=len(class_names))
).to(device)

In [103]:
for param in model.fc.parameters():
    param.requires_grad = True

In [104]:
from torchinfo import summary

summary(model=model.to(device),
        input_size=(1, 3, 288, 288),
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"],
        device=device)

Layer (type (var_name))                  Input Shape          Output Shape         Param #              Trainable
ResNet (ResNet)                          [1, 3, 288, 288]     [1, 4]               --                   Partial
├─Conv2d (conv1)                         [1, 3, 288, 288]     [1, 64, 144, 144]    (9,408)              False
├─BatchNorm2d (bn1)                      [1, 64, 144, 144]    [1, 64, 144, 144]    (128)                False
├─ReLU (relu)                            [1, 64, 144, 144]    [1, 64, 144, 144]    --                   --
├─MaxPool2d (maxpool)                    [1, 64, 144, 144]    [1, 64, 72, 72]      --                   --
├─Sequential (layer1)                    [1, 64, 72, 72]      [1, 256, 72, 72]     --                   False
│    └─Bottleneck (0)                    [1, 64, 72, 72]      [1, 256, 72, 72]     --                   False
│    │    └─Conv2d (conv1)               [1, 64, 72, 72]      [1, 64, 72, 72]      (4,096)              False
│    │    

In [105]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [106]:
import engine

results = engine.train(model=model,
                       train_dataloader=train_dataloader,
                       test_dataloader=test_dataloader,
                       loss_fn=loss_fn,
                       optimizer=optimizer,
                       epochs=5,
                       device=device)

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 1/5, Train Loss: 1.2369, Train Acc: 0.4402, Test Loss: 1.0304, Test Acc: 0.5403
Epoch: 2/5, Train Loss: 1.1014, Train Acc: 0.5428, Test Loss: 0.9675, Test Acc: 0.5827
Epoch: 3/5, Train Loss: 1.0385, Train Acc: 0.5730, Test Loss: 0.9177, Test Acc: 0.5792
Epoch: 4/5, Train Loss: 0.9999, Train Acc: 0.5833, Test Loss: 0.8983, Test Acc: 0.5869


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x000001CCB9210860>
Traceback (most recent call last):
  File "C:\Users\sandu\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\data\dataloader.py", line 1618, in __del__
    self._shutdown_workers()
  File "C:\Users\sandu\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\data\dataloader.py", line 1576, in _shutdown_workers
    if self._persistent_workers or self._workers_status[worker_id]:
                                   ^^^^^^^^^^^^^^^^^^^^
AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute '_workers_status'


Epoch: 5/5, Train Loss: 0.9712, Train Acc: 0.5935, Test Loss: 0.8803, Test Acc: 0.5992
